# In this notebook we look into the performance of the Sinkhorn algorithm.

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
%matplotlib inline
%load_ext autoreload
%autoreload 

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/Sinkhorn_images'):
    os.makedirs('../Images/Sinkhorn_images')

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors,  c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
import computational_OT

## Entropy regularized formulation

The primal entropy regularized formulation of OT is given by:
$$
OT_{\epsilon}(\alpha,\beta) = min_{\pi \in \mathcal{U}(\alpha,\beta)} \langle C,\pi \rangle +\epsilon KL(\pi\|\alpha \otimes \beta)\ ,
$$
where
$\ 
KL(\pi\|\alpha \otimes \beta) 
\ $ is the KL-divergence and $\ \mathcal{U}(\alpha,\beta)=\{\pi: \pi\mathcal{1}=\alpha, \pi^{T}\mathcal{1}=\beta\}$. 

## Sinkhorn 
The optimal coupling $\pi^{*}$ has the following form :
$$
\pi^{*} = \alpha \odot diag(u)K diag(v)\odot \beta
$$
and we know that $\pi^{*}\mathbb{1}=\alpha$ and $(\pi^{*})^{T}\mathbb{1}=\beta$.
###
Therefore, Sinkhorn updates is given by the following iterative projections
$$
u^{t+1}  \leftarrow \frac{1}{K(v^{t}\odot \beta)}\ , \ 
v^{t+1}  \leftarrow \frac{1}{K^{T}(u^{t+1}\odot \alpha)}\ , 
$$
where 
$K = e^{-\frac{C}{\epsilon}}\in M_{n\times m}(\mathbb{R}),\ \alpha \in \mathbb{R}^{n},\ \beta \in \mathbb{R}^{m}\ ,\ u\in \mathbb{R}^{n},\ v\in \mathbb{R}^{m}\ and \ (u^{0},v^{0})=(u,v)\ .$



### I. Sinkhorn for varying N and fixed $\varepsilon$

In [ ]:
def randomsampledata( N ):
  x = []
  y = []
  N = np.sort( N )
  for i in range(len(N)):
    x.append( np.random.rand( 2, N[i] ) - 0.5 )
    theta = 2 * np.pi * np.random.rand( 1, N[i] )
    r = 0.8 + 0.2 * np.random.rand( 1, N[i] )
    y.append( np.vstack( ( np.cos(theta) * r, np.sin(theta) * r ) ) )
  
  return x, y, N

In [ ]:
N = [ 200, 400, 600, 800, 1000 ]
x, y, N = randomsampledata( N )

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")  
#Epsilon
epsilon = .06
SinkhornP = []
results_Sinkhorn = []
times_Sinkhorn = []
for i in range(len(N)):
  print( "Doing for ", N[i] )
  xi, yi = x[i], y[i]
  #Cost matrix
  C = distmat( xi, yi )
  # a and b
  a = normalize( np.ones( N[i] ) )
  b = normalize( np.ones( N[i] ) )
  #Kernel
  K = np.exp( - C/epsilon )
  print( " |- Iterating" )
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          epsilon )
  out = Optimizer._update()
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/epsilon )
  K = np.exp( - C/epsilon )
  v_opt =  np.exp( out['potential_g']/epsilon )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP.append( P_opt )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range(len(results_Sinkhorn)):
  error = np.asarray( results_Sinkhorn[i]['error_a'] ) + np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error, label = 'Sinkhorn for $\epsilon = $'+ str(epsilon), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend( [ "N = "+str(i) for i in N ], loc = "upper right" )
plt.savefig( "../Images/Sinkhorn_images/ConvergenceSinkhorn.pdf", format = 'pdf'  )
plt.show()

#### Objective function plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Objective Function" )
for result in results_Sinkhorn:
  plt.plot( np.asarray( result['objective_values'] ).flatten(), linewidth = 2 )
# end for
plt.legend( [ "N = "+str(i) for i in N ], loc = "upper right" )
plt.savefig( "../Images/Sinkhorn_images/ObjectivefunctionSinkhorn.pdf", format = 'pdf'  )
plt.show()

### II. Sinkhron for varying $\varepsilon$

In [ ]:
N = [ 400, 500 ]

In [ ]:
x = np.random.rand( 2, N[0] ) - 0.5
theta = 2 * np.pi * np.random.rand( 1, N[1] )
r = 0.8 + .2 * np.random.rand( 1, N[1] )
y = np.vstack( ( r * np.cos( theta ), r * np.sin( theta ) ) )

In [ ]:
# Sinkhorn
print("Sinkhorn.... ")
print( "Doing for (",N[0], N[1],")." )
SinkhornP = []
results_Sinkhorn = []
times_Sinkhorn = []
epsilons = [ 1.0, 0.5, 0.1,  0.05, 0.01, 0.001 ]
#Cost matrix
C = distmat( x, y )
# a and b
a = normalize( np.ones( N[0] ) )
b = normalize( np.ones( N[1] ) )
for eps in epsilons:
  print( "For epsilon = "+str(eps)+":" )    
  #Kernel
  K = np.exp( - C/eps )
  print( " |- Iterating" )
  #Inflating
  u = a
  v = b
  start = time.time()
  Optimizer = computational_OT.sinkhorn(  K,
                                          a,
                                          b,
                                          u,
                                          v,
                                          eps )
  out = Optimizer._update( max_iterations = 500 )
  results_Sinkhorn.append( out )
  end = time.time()
  times_Sinkhorn.append( end - start )
  print( " |- Computing P" )
  print( "" )
  u_opt = np.exp( out['potential_f']/eps )
  K = np.exp( - C/eps )
  v_opt =  np.exp( out['potential_g']/eps )
  P_opt = GetP( u_opt, K, v_opt )
  SinkhornP.append( P_opt )
# end for

#### Error plot

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "$||P1 -a||_1+||P1 -b||_1$" )
for i in range( len(results_Sinkhorn) ):
  error = np.asarray( results_Sinkhorn[i]['error_a'] ) + np.asarray( results_Sinkhorn[i]['error_b'] )
  plt.plot( error, label='Sinkhorn for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
# end for
plt.yscale( 'log' )
plt.legend( loc = "upper right" )
plt.savefig( "../Images/Sinkhorn_images/ConvergenceSinkhornvaryingepsilon.pdf", format = 'pdf'  )
plt.show()